In [1]:
import tensorflow as tf
import numpy as np
import time, os, logging
import sample_tf2, model_tf2, encoder

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
CHECKPOINT_ROOT = './checkpoint'
SEQ_LEN = 1024

In [6]:
gpt2 = model_tf2.GPT2(model_tf2.HPARAMS['117M'])

In [7]:
X = tf.convert_to_tensor(np.array([[35, 789], [98, 69]]))
# mask = model_tf2.create_look_ahead_mask(2)

In [8]:
logits, presents, _ = gpt2(X, None)

In [9]:
gpt2.summary()

Model: "gpt2_tf2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  52511744  
_________________________________________________________________
decoder (Decoder)            multiple                  302311424 
_________________________________________________________________
look_up (LookUp)             multiple                  51463168  
Total params: 354,823,168
Trainable params: 354,823,168
Non-trainable params: 0
_________________________________________________________________


In [10]:
print("Total {} trainable variables.".format(
    len(gpt2.trainable_variables)))
for v in gpt2.trainable_variables:
    print(v.name, v.shape, v.dtype)

Total 292 trainable variables.
gpt2_tf2/wpe:0 (1024, 1024) <dtype: 'float32'>
gpt2_tf2/wte:0 (50257, 1024) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_attn/kernel:0 (1024, 3072) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_attn/bias:0 (3072,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_proj/kernel:0 (1024, 1024) <dtype: 'float32'>
gpt2_tf2/decoder/h0/attn/c_proj/bias:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_fc/kernel:0 (1024, 4096) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_fc/bias:0 (4096,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_proj/kernel:0 (4096, 1024) <dtype: 'float32'>
gpt2_tf2/decoder/h0/mlp/c_proj/bias:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_1/gamma:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_1/beta:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_2/gamma:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h0/ln_2/beta:0 (1024,) <dtype: 'float32'>
gpt2_tf2/decoder/h1/attn/c_attn/kernel:0 (1024, 3072) <dtype: 'float32'>
gpt2_tf2/decode

## Load from TF1 Checkpoint

In [11]:
ckpt_directory = "../models/355M"

In [12]:
ckpt_vars = tf.train.list_variables(ckpt_directory)
names = []
tensors = []

for name, shape in ckpt_vars:
    print("Loading TF weight {} with shape {}".format(name, shape))
    tensor = tf.train.load_variable(ckpt_directory, name)
    names.append(str(name[6:].split("/")))
    tensors.append(tensor)
assert len(names) == len(tensors)
print("{} vars loaded from ckpt {}".format(len(ckpt_vars), ckpt_directory))

Loading TF weight model/h0/attn/c_attn/b with shape [3072]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h0/attn/c_proj/b with shape [1024]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h0/ln_1/b with shape [1024]
Loading TF weight model/h0/ln_1/g with shape [1024]
Loading TF weight model/h0/ln_2/b with shape [1024]
Loading TF weight model/h0/ln_2/g with shape [1024]
Loading TF weight model/h0/mlp/c_fc/b with shape [4096]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 1024, 4096]
Loading TF weight model/h0/mlp/c_proj/b with shape [1024]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/h1/attn/c_attn/b with shape [3072]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h1/attn/c_proj/b with shape [1024]
Loading TF weight model/h1/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h1/ln_1/b with

Loading TF weight model/h2/attn/c_attn/b with shape [3072]
Loading TF weight model/h2/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h2/attn/c_proj/b with shape [1024]
Loading TF weight model/h2/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h2/ln_1/b with shape [1024]
Loading TF weight model/h2/ln_1/g with shape [1024]
Loading TF weight model/h2/ln_2/b with shape [1024]
Loading TF weight model/h2/ln_2/g with shape [1024]
Loading TF weight model/h2/mlp/c_fc/b with shape [4096]
Loading TF weight model/h2/mlp/c_fc/w with shape [1, 1024, 4096]
Loading TF weight model/h2/mlp/c_proj/b with shape [1024]
Loading TF weight model/h2/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/h20/attn/c_attn/b with shape [3072]
Loading TF weight model/h20/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h20/attn/c_proj/b with shape [1024]
Loading TF weight model/h20/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h20/ln_1/b

Loading TF weight model/h9/mlp/c_proj/b with shape [1024]
Loading TF weight model/h9/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/ln_f/b with shape [1024]
Loading TF weight model/ln_f/g with shape [1024]
Loading TF weight model/wpe with shape [1024, 1024]
Loading TF weight model/wte with shape [50257, 1024]
292 vars loaded from ckpt ../models/355M


In [13]:
ckpt_vmap = dict(zip(names, tensors))

In [14]:
vname_mapping = {"kernel": "w", "bias": "b", "gamma": "g", "beta": "b"}

In [15]:
for v in gpt2.trainable_variables:
    tf2_vname = v.name[9:-2].split("/")
    if tf2_vname[0] == "decoder":
        tf2_vname = tf2_vname[1:]
    if tf2_vname[-1] in vname_mapping:
        tf2_vname[-1] = vname_mapping[tf2_vname[-1]]
    tf2_vvalue = np.squeeze(ckpt_vmap[str(tf2_vname)])
    assert v.shape == tf2_vvalue.shape, \
        "{} has different shape: gpt2_tf2 {} vs gpt2_tf1 {}" \
        .format(v.name, str(v.shape), str(tf2_vvalue.shape))
    v.assign(tf2_vvalue)

## Test loaded gpt2_tf2 model

In [16]:
enc = encoder.get_encoder("355M", "../models")

In [17]:
context = enc.encode("Can we be friends?")
context = tf.convert_to_tensor(context,dtype=tf.int32)
context = tf.expand_dims(context, axis=0)

In [18]:
output = sample_tf2.sample_sequence(
    gpt2_model=gpt2,
    length=200,
    context=context,
    top_p=0,
    top_k=40,
    batch_size=1
)

In [19]:
enc.decode(output[0].numpy())

'Can we be friends? How do you make someone fall in love, and do you have to be a monster to get someone to fall in love?<|endoftext|>Saskatoon police officers are appealing for help finding a young man who left his girlfriend\'s home with her gun tucked into her purse before she killed herself.\n\nBarek Hulak and his girlfriend, 20-year-old Melissa Marie Hulak, were seen leaving their home in the 12500 block of West 37th Street at around 2:30 a.m. Tuesday. Police say they called 911 in the hours following the attack but did not immediately receive an answer.\n\nA video of the 911 call shows Hulak walking toward the front door, putting her phone into her hand, and walking back into the home. Police have called her a "preppy" and "caring" girl.\n\nHulak had texted Hulak and her boyfriend, saying she couldn\'t handle being a mother or being apart from her'

## Save model

In [20]:
gpt2.save_weights("../models/355M_tf2/pretrained_weights.h5")

---
# Load the saved model

In [21]:
import tensorflow as tf
import numpy as np
import time, os, logging
import sample_tf2, model_tf2, encoder

In [11]:
gpt2_loaded = model_tf2.GPT2(model_tf2.HPARAMS['117M'])

In [12]:
X = tf.convert_to_tensor(np.array([[35, 789], [98, 69]]))

In [13]:
logits, presents, _ = gpt2_loaded(X, None)

In [14]:
gpt2_loaded.load_weights("../models/117M_tf2/pretrained_weights.h5")

In [15]:
gpt2_loaded.summary()

Model: "gpt2_tf2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  39383808  
_________________________________________________________________
decoder (Decoder)            multiple                  85056000  
_________________________________________________________________
look_up_1 (LookUp)           multiple                  38597376  
Total params: 124,439,808
Trainable params: 124,439,808
Non-trainable params: 0
_________________________________________________________________


## Test loaded gpt2_tf2 model

In [16]:
enc = encoder.get_encoder("117M", "../models")

In [36]:
context = enc.encode("Can we be friends?")
context = tf.convert_to_tensor(context,dtype=tf.int32)
context = tf.expand_dims(context, axis=0)

In [37]:
output = sample_tf2.sample_sequence(
    gpt2_model=gpt2_loaded,
    length=200,
    context=context,
    top_p=0,
    top_k=40,
    batch_size=1
)

In [38]:
enc.decode(output[0].numpy())

"Can we be friends?\n\n\nYou may use this topic\n\nCheck out my website's the best way to learn.\n\nA lot has to do\n\nWant to make a life change? The Completely do.\n\nHow to become a better man!\n\nJust get a job to earn a living now.\n\nYour life's only to take you on.\n\nDo you want to be a better man?\n\nIf you don't want to lose that you're a better than ever before\n\nGo to the right people now.\n\nFind the help right now.\n\nDo you want to go to a job for help?\n\nA lot happens before we're ready to go.\n\nDo you want to stay on what you are now\n\nYou are ready.\n\nDid you know the most important thing to know from being a better man (and a better man than you) is...\n\nNot getting involved in politics, becoming a better than"

In [30]:
inp_text = "How are you my friend? I am here to chat with you. I like your shirt"

In [31]:
inp = tf.convert_to_tensor(
            np.stack(
                [enc.encode(inp_text)]),
            dtype=tf.int32
        )
inp

<tf.Tensor: shape=(1, 18), dtype=int32, numpy=
array([[ 2437,   389,   345,   616,  1545,    30,   314,   716,   994,
          284,  8537,   351,   345,    13,   314,   588,   534, 10147]],
      dtype=int32)>

In [32]:
predictions, _, _ = gpt2_loaded(inp, None)

In [33]:
enc.decode(np.argmax(predictions[0,:-1].numpy(), axis=1))

',/._ of\n,,tic the the\n won. is in name'

In [34]:
output = sample_tf2.sample_sequence(
    gpt2_model=gpt2_loaded,
    length=200,
    context=inp,
    top_p=0,
    top_k=40,
    batch_size=1
)

In [35]:
enc.decode(output[0].numpy())

"How are you my friend? I am here to chat with you. I like your shirton\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDont'\n\n\n\nyou can do better in the best way to\n\n\ntell me\n\n\nI could hear ya\n\nMy name is\n\nWe can tell you how i mean\n\nYou get what I mean. I like doing\n\nI can't understand\n\nYou make me? What i'm say\n\nI'm not doing well,\n\n\nI'm not in my mouth to\n\n\nI'm not\n\n\nOh\n\n\nyou can I have\n\n\n\nI have\n\n\n\n\nyou\n\n\n\nI like a\n\n\n\n\n\n\nI've been\n\n\n\nwhat you\n\n\n\n\n\n\n\n\n\nI'm\n\n\n\n\n\nyou\n\n\n\nwhat you can\n\n\n\n\nI'm of\n\n\n\n\nyou\n\nyou"